In [1]:
# Import required libraries

# eda tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# model preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# feature selection
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# algorithms
from statsmodels.api import OLS
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# evaluation
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Read data
inspections_df = pd.read_csv('data/food_inspections.csv', parse_dates=['Inspection Date'])

inspections_df.dropna(subset='License #', inplace=True)

inspections_df['License #'].astype('int32')

# Display sample data
inspections_df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2579974,PING PONG,PING PONG,1765705.0,Restaurant,Risk 1 (High),3322-3326 N BROADWAY,CHICAGO,IL,60657.0,2023-08-11,Non-Inspection,No Entry,NaN,41.942623,-87.644625,"(41.94262315814179, -87.64462536460103)"
1,2579958,SOUTH SHORE FOOD MART INC.,SOUTH SHORE FOOD MART,2340833.0,Grocery Store,Risk 1 (High),7900 S SOUTH SHORE DR,CHICAGO,IL,60617.0,2023-08-11,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.751856,-87.548250,"(41.75185560246702, -87.54825033733893)"
2,2579936,GREAT LAKES ACADEMY,GREAT LAKES ACADEMY,3305802.0,School,Risk 1 (High),8401 S Saginaw AVE,CHICAGO,IL,60617.0,2023-08-11,Canvass Re-Inspection,Pass,NaN,41.742635,-87.559870,"(41.74263536816081, -87.55986972603321)"
3,2579926,LUCKY STRIKE DOWNTOWN CHICAGO,LUCKY STRIKE DOWNTOWN CHICAGO,2917711.0,NaN,Risk 3 (Low),322 E ILLINOIS ST,CHICAGO,IL,60611.0,2023-08-10,License Re-Inspection,Pass,NaN,41.891133,-87.619160,"(41.891133170854566, -87.61916020704619)"
4,2579927,LUCKY STRIKE DOWNTOWN CHICAGO,LUCKY STRIKE DOWNTOWN CHICAGO,2917712.0,NaN,Risk 3 (Low),322 E ILLINOIS ST,CHICAGO,IL,60611.0,2023-08-10,License Re-Inspection,Pass,NaN,41.891133,-87.619160,"(41.891133170854566, -87.61916020704619)"


In [3]:
inspections_df.dtypes

Inspection ID               int64
DBA Name                   object
AKA Name                   object
License #                 float64
Facility Type              object
Risk                       object
Address                    object
City                       object
State                      object
Zip                       float64
Inspection Date    datetime64[ns]
Inspection Type            object
Results                    object
Violations                 object
Latitude                  float64
Longitude                 float64
Location                   object
dtype: object

In [4]:
len(inspections_df)

258045

In [5]:
temp = inspections_df['Facility Type'].value_counts()

In [6]:
temp.head(50)

Restaurant                           173206
Grocery Store                         32388
School                                16222
Children's Services Facility           5250
Bakery                                 3738
Daycare Above and Under 2 Years        3407
Daycare (2 - 6 Years)                  3083
Long Term Care                         1863
Catering                               1594
Liquor                                 1106
Mobile Food Dispenser                   923
Mobile Food Preparer                    911
Hospital                                764
Golden Diner                            723
Daycare Combo 1586                      691
Wholesale                               552
TAVERN                                  389
Shared Kitchen User (Long Term)         314
Daycare (Under 2 Years)                 312
Special Event                           286
Shared Kitchen                          219
GAS STATION                             198
BANQUET HALL                    

To be thorough, any food related business should be added including multipurpose businesses like 'Grocery/Restaurant'.  For simplicity, I will only use restaurant data.

In [7]:
restaurant_df = inspections_df[inspections_df['Facility Type'] == 'Restaurant'].copy()
restaurant_df.drop(['DBA Name', 'Location'], axis=1, inplace=True)
restaurant_df.head()

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
0,2579974,PING PONG,1765705.0,Restaurant,Risk 1 (High),3322-3326 N BROADWAY,CHICAGO,IL,60657.0,2023-08-11,Non-Inspection,No Entry,NaN,41.942623,-87.644625
5,2579896,LAS ISLAS MARIAS,1943409.0,Restaurant,Risk 1 (High),2400 S PULASKI RD,CHICAGO,IL,60623.0,2023-08-10,Canvass Re-Inspection,Fail,NaN,41.847858,-87.724795
6,2579853,Lito's Taco and Empanada House,2762609.0,Restaurant,Risk 1 (High),2460 N CLARK ST,CHICAGO,IL,60614.0,2023-08-09,Canvass Re-Inspection,No Entry,NaN,41.927067,-87.641530
7,2579910,SHU TIN XIA,2574892.0,Restaurant,Risk 1 (High),2428-2436 S WALLACE ST,CHICAGO,IL,60616.0,2023-08-09,Canvass,Out of Business,NaN,41.848386,-87.641960
8,2579839,QUERETACO,2918599.0,Restaurant,Risk 2 (Medium),2247 E 71ST ST,CHICAGO,IL,60649.0,2023-08-09,License,Pass,NaN,41.766032,-87.569550


In [8]:
restaurant_df.dtypes

Inspection ID               int64
AKA Name                   object
License #                 float64
Facility Type              object
Risk                       object
Address                    object
City                       object
State                      object
Zip                       float64
Inspection Date    datetime64[ns]
Inspection Type            object
Results                    object
Violations                 object
Latitude                  float64
Longitude                 float64
dtype: object

In [9]:
business_info = pd.read_csv('data/all_licensed_businesses.csv')
business_info.drop(['ID', 'LICENSE STATUS', 'ACCOUNT NUMBER', ], axis=1, inplace=True)
business_info.head()

C:\Users\ejwda\AppData\Local\Temp\ipykernel_26620\4274237581.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  business_info = pd.read_csv('data/all_licensed_businesses.csv')


,LICENSE ID,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,WARD,PRECINCT,...,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,76522,1,THORNDALE CONSTRUCTION,THORNDALE CONSTRUCTION,11243 CHESAPEAKE PLAC 1ST,WESTCHESTER,IL,60154,NaN,NaN,...,N,02/16/2000,02/15/2001,12/22/2003,08/24/2009,NaN,NaN,NaN,NaN,NaN
1,119268,1,"PAT HAMILTON, INC",PAT HAMILTON CO.,17021 S MAGNOLIA DR 1ST,HAZEL CREST,IL,60429,NaN,NaN,...,N,02/16/1996,02/15/1997,12/22/2003,04/12/2006,NaN,NaN,NaN,NaN,NaN
2,2479692,2,"AMERICAN ARBITRATION ASSOCIATION, INC.","AMERICAN ARBITRATION ASSOCIATION, INC.",150 N MICHIGAN AVE 30TH 3050,CHICAGO,IL,60601,34.0,9.0,...,N,09/16/2016,09/15/2018,08/29/2016,08/30/2016,NaN,NaN,41.884531,-87.624602,"(41.88453096378336, -87.62460204460082)"
3,2375632,1,MEE INC,MEE INC,11845 W LARAWAY RD 1,FRANKFORT,IL,60423,NaN,NaN,...,N,03/16/2015,03/15/2017,03/12/2015,03/13/2015,NaN,NaN,NaN,NaN,NaN
4,1940503,45,"FEDEX OFFICE AND PRINT SERVICES, INC.","FEDEX OFFICE AND PRINT SVCS, INC.",2301 S DR MARTIN LUTHER KING JR DR SOUTH BL,CHICAGO,IL,60616,4.0,3.0,...,N,01/16/2009,01/15/2011,01/08/2009,01/22/2009,NaN,NaN,NaN,NaN,NaN


In [10]:
# Find features that are of little value
business_info.isna().sum()/len(business_info)*100

LICENSE ID                            0.000000
SITE NUMBER                           0.000000
LEGAL NAME                            0.000000
DOING BUSINESS AS NAME                0.017125
ADDRESS                               0.000000
CITY                                  0.000270
STATE                                 0.001082
ZIP CODE                              0.026588
WARD                                  7.501345
PRECINCT                             12.621483
WARD PRECINCT                         7.499002
POLICE DISTRICT                      12.044837
LICENSE CODE                          0.000000
LICENSE DESCRIPTION                   0.000000
BUSINESS ACTIVITY ID                 48.954909
BUSINESS ACTIVITY                    48.954909
LICENSE NUMBER                        0.000090
APPLICATION TYPE                      0.000000
APPLICATION CREATED DATE             76.708958
APPLICATION REQUIREMENTS COMPLETE     1.654047
PAYMENT DATE                          2.015645
CONDITIONAL A

In [11]:
business_info['CONDITIONAL APPROVAL'].value_counts()

N    1105249
Y       4272
Name: CONDITIONAL APPROVAL, dtype: int64

In [12]:
business_info['APPLICATION TYPE'].value_counts()

RENEW     844170
ISSUE     248420
C_LOC      14958
C_EXPA       811
C_SBA        661
C_CAPA       501
Name: APPLICATION TYPE, dtype: int64

In [13]:
# keep only pertinent columns
business_info = business_info[['DOING BUSINESS AS NAME','LICENSE ID', 'ADDRESS', 'WARD', 'PRECINCT', 'WARD PRECINCT', 'POLICE DISTRICT', 'LICENSE CODE', 'LICENSE DESCRIPTION', 'LICENSE NUMBER','BUSINESS ACTIVITY ID', 'BUSINESS ACTIVITY', 
               'APPLICATION TYPE','APPLICATION REQUIREMENTS COMPLETE', 'CONDITIONAL APPROVAL', 'LICENSE TERM START DATE', 
               'LICENSE TERM EXPIRATION DATE', 'LICENSE APPROVED FOR ISSUANCE', 'DATE ISSUED']]

In [14]:
# remove unknown Application Type
business_info = business_info[~business_info['APPLICATION TYPE'].isin(['C_LOC', 'C_SBA', 'C_EXPA', 'C_CAPA'])]

### Check to see how many matching business id's there are  
I would like to merge the datasets to obtain renewal/issue info and a few other features

In [15]:
# License # and LICENSE NUMBER are equivalent

restaurant_licenses = restaurant_df['License #'].unique()
business_licenses = business_info['LICENSE NUMBER'].unique()

print(f'Restaurant licenses: {len(restaurant_licenses)}')
print(f'Business licenses: {len(business_licenses)}')


compare=[]
for i in restaurant_licenses:
    compare.append(i in business_licenses)
    
print(f'Matches:  {len(restaurant_licenses[compare])} ({len(restaurant_licenses[compare])/len(restaurant_licenses)*100:0.0f}%)')


Restaurant licenses: 24083
Business licenses: 322425
Matches:  23336 (97%)


### Check a common Restaurant

In [16]:
restaurant_df[restaurant_df['AKA Name'] == 'SUBWAY'].sort_values(by='License #', ascending=True)

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
94598,2300450,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-28,Canvass,Out of Business,NaN,41.968506,-87.688338
8406,2293545,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-13,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.968506,-87.688338
94955,2293900,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-20,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968506,-87.688338
224619,519380,SUBWAY,15059.0,Restaurant,Risk 1 (High),1313 W TOUHY AVE,CHICAGO,IL,60626.0,2012-02-03,Canvass Re-Inspection,Pass,NaN,42.012710,-87.664034
115695,1955999,SUBWAY,15059.0,Restaurant,Risk 1 (High),1313 W TOUHY AVE,CHICAGO,IL,60626.0,2017-11-14,Complaint,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,42.012710,-87.664034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37107,2570591,SUBWAY,2890832.0,Restaurant,Risk 1 (High),2620 N NARRAGANSETT AVE,CHICAGO,IL,60639.0,2023-02-02,License,Pass,NaN,41.927995,-87.785752
38592,2573846,SUBWAY,2900790.0,Restaurant,Risk 1 (High),400 N ORLEANS ST,CHICAGO,IL,60654.0,2023-04-07,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.889172,-87.637110
235,2577597,SUBWAY,2906313.0,Restaurant,Risk 1 (High),1958 W PETERSON AVE,CHICAGO,IL,60660.0,2023-06-21,License,Pass,NaN,41.990792,-87.679365
35710,2577541,SUBWAY,2917457.0,Restaurant,Risk 1 (High),1252 S HALSTED ST,CHICAGO,IL,60607.0,2023-06-20,License,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.865599,-87.647018


In [17]:
restaurant_df.dtypes

Inspection ID               int64
AKA Name                   object
License #                 float64
Facility Type              object
Risk                       object
Address                    object
City                       object
State                      object
Zip                       float64
Inspection Date    datetime64[ns]
Inspection Type            object
Results                    object
Violations                 object
Latitude                  float64
Longitude                 float64
dtype: object

In [18]:
restaurant_df['year'] = restaurant_df['Inspection Date'].dt.strftime('%Y')

In [19]:
business_info[business_info['DOING BUSINESS AS NAME'] == 'SUBWAY'].sort_values(by='LICENSE NUMBER', ascending=True).iloc[:,0:20]

,DOING BUSINESS AS NAME,LICENSE ID,ADDRESS,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
478480,SUBWAY,2745431,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2020,N,10/16/2020,10/15/2022,09/23/2020,09/24/2020
515280,SUBWAY,1849583,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,09/21/2007,N,11/16/2007,10/15/2008,10/29/2007,12/05/2007
810465,SUBWAY,1279771,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,09/24/2002,N,11/16/2002,11/15/2003,11/04/2002,11/05/2002
74404,SUBWAY,1427126,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,09/26/2003,N,11/16/2003,11/15/2004,12/22/2003,01/12/2004
390466,SUBWAY,2050480,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/16/2010,N,10/16/2010,10/15/2012,11/03/2010,11/04/2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444962,SUBWAY,2900790,400 N ORLEANS ST 1 1,42.0,17.0,42-17,18.0,1006,Retail Food Establishment,2900790.0,781,Sale of Food Prepared Onsite With Dining Area,ISSUE,03/24/2023,N,04/11/2023,04/15/2025,04/10/2023,04/11/2023
250559,SUBWAY,2906313,1958 W PETERSON AVE 1ST,40.0,9.0,40-9,24.0,1006,Retail Food Establishment,2906313.0,781,Sale of Food Prepared Onsite With Dining Area,ISSUE,04/25/2023,N,07/26/2023,08/15/2025,07/26/2023,07/26/2023
275314,SUBWAY,2911663,780 N MILWAUKEE AVE,27.0,NaN,27-,NaN,1006,Retail Food Establishment,2911663.0,735,Preparation of Food and Dining on Premise With...,ISSUE,05/16/2023,N,08/16/2023,09/15/2025,08/15/2023,08/16/2023
249948,SUBWAY,2917457,1252 S HALSTED ST 1ST 5,28.0,7.0,28-7,12.0,1006,Retail Food Establishment,2917457.0,781,Sale of Food Prepared Onsite With Dining Area,ISSUE,06/15/2023,N,07/26/2023,08/15/2025,07/26/2023,07/26/2023


### Check other records to ensure matching by decomals on LICENSE NUMBER works

In [20]:
business_info[business_info['LICENSE NUMBER'] == 15059.0]

,DOING BUSINESS AS NAME,LICENSE ID,ADDRESS,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
5512,SUBWAY,2863846,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2022,N,10/16/2022,10/15/2024,08/16/2022,08/17/2022
74404,SUBWAY,1427126,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,09/26/2003,N,11/16/2003,11/15/2004,12/22/2003,01/12/2004
262450,SUBWAY,2350763,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2014,N,10/16/2014,10/15/2016,02/28/2015,03/03/2015
275341,SUBWAY,2180286,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2012,N,10/16/2012,10/15/2014,10/05/2012,10/09/2012
291037,SUBWAY,2617872,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2018,N,10/16/2018,10/15/2020,09/19/2018,09/20/2018
325411,SUBWAY,1745013,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,09/20/2006,N,11/16/2006,11/15/2007,11/14/2006,11/15/2006
332112,SUBWAY,1624443,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,09/22/2005,N,11/16/2005,11/15/2006,11/01/2005,11/02/2005
390466,SUBWAY,2050480,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/16/2010,N,10/16/2010,10/15/2012,11/03/2010,11/04/2010
460990,SUBWAY,1923392,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2008,N,10/16/2008,10/15/2010,10/06/2008,10/07/2008
478480,SUBWAY,2745431,1313 W TOUHY AVE,49.0,27.0,49-27,24.0,1006,Retail Food Establishment,15059.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2020,N,10/16/2020,10/15/2022,09/23/2020,09/24/2020


### See if the ADDRESS matches

In [21]:
business_info[business_info['ADDRESS']=='4771 N LINCOLN AVE']

,DOING BUSINESS AS NAME,LICENSE ID,ADDRESS,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
121543,SUBWAY,2677482,4771 N LINCOLN AVE,47.0,NaN,47-,NaN,1006,Retail Food Establishment,2677482.0,781,Sale of Food Prepared Onsite With Dining Area,ISSUE,06/25/2019,N,07/24/2019,08/15/2021,07/22/2019,07/24/2019
416655,CAFE DESCARTES,1646813,4771 N LINCOLN AVE,47.0,20.0,47-20,19.0,1006,Retail Food Establishment,1646813.0,775,Retail Sales of Perishable Foods,ISSUE,10/21/2005,N,11/17/2005,11/15/2006,11/16/2005,11/17/2005
859424,CAFE DESCARTES,1747076,4771 N LINCOLN AVE,47.0,20.0,47-20,19.0,1006,Retail Food Establishment,1646813.0,775,Retail Sales of Perishable Foods,RENEW,09/20/2006,N,11/16/2006,11/15/2007,12/19/2006,12/19/2006
920645,SUBWAY,2796097,4771 N LINCOLN AVE,47.0,NaN,47-,NaN,1006,Retail Food Establishment,2677482.0,781,Sale of Food Prepared Onsite With Dining Area,RENEW,06/15/2021,N,08/16/2021,08/15/2023,01/31/2022,09/07/2022


### Check range of License \#

In [22]:
restaurant_df['License #'].describe()

count    1.732060e+05
mean     1.757982e+06
std      8.773912e+05
min      0.000000e+00
25%      1.473493e+06
50%      2.048650e+06
75%      2.379895e+06
max      3.624677e+06
Name: License #, dtype: float64

In [23]:
restaurant_df['License #'].value_counts()

1354323.0    196
0.0          158
1574001.0     69
60184.0       60
1974745.0     58
            ... 
2827305.0      1
2771129.0      1
2293833.0      1
2283741.0      1
1800100.0      1
Name: License #, Length: 24083, dtype: int64

In [24]:
restaurant_df.columns

Index(['Inspection ID', 'AKA Name', 'License #', 'Facility Type', 'Risk',
       'Address', 'City', 'State', 'Zip', 'Inspection Date', 'Inspection Type',
       'Results', 'Violations', 'Latitude', 'Longitude', 'year'],
      dtype='object')

In [25]:
restaurant_df.groupby(['Address','AKA Name', 'License #','year']).count()['Risk']

Address              AKA Name                    License #  year
                     CASA CENTRAL LA POSADA      0.0        2010    1
                     EAT N RUN CHICKEN AND FISH  2442868.0  2016    1
1 E 83RD ST          SUBWAY                      1938179.0  2010    1
                                                            2011    4
                                                            2012    1
                                                                   ..
9956 S WESTERN AVE   FOX'S BEVERLY PUB           149.0      2019    2
                                                            2020    2
                                                            2021    2
                                                            2022    1
                                                            2023    2
Name: Risk, Length: 106780, dtype: int64

In [26]:
restaurant_df[restaurant_df['AKA Name'] == 'SUBWAY'].groupby(['Address','AKA Name', 'License #','year']).count()['Risk']

Address             AKA Name  License #  year
1 E 83RD ST         SUBWAY    1938179.0  2010    1
                                         2011    4
                                         2012    1
                                         2013    3
                                         2014    2
                                                ..
9518 S HALSTED ST   SUBWAY    1846937.0  2019    1
953 W WEBSTER AVE   SUBWAY    2327923.0  2014    3
                              2354661.0  2014    1
                                         2015    1
                                         2016    2
Name: Risk, Length: 2652, dtype: int64

In [27]:
bad_license_numbers = restaurant_df[restaurant_df['License #'] < 100]
len(bad_license_numbers)

247

In [28]:
bad_license_numbers

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,year
332,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.711605,-87.535133,2023
744,2573601,PENNINGTON'S FOOD AND SNACKS,0.0,Restaurant,Risk 1 (High),410 E 79TH ST,CHICAGO,IL,60619.0,2023-04-03,Complaint,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.751234,-87.614471,2023
2859,2553296,KRISPY KREME DOUGHNUTS,0.0,Restaurant,Risk 2 (Medium),108 N STATE ST,CHICAGO,IL,60602.0,2022-03-23,Canvass,Out of Business,NaN,41.883423,-87.628022,2022
4962,2484249,XANDO COFFEE & BAR / COSI SANDWICH BAR,9.0,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,CHICAGO,IL,60603.0,2021-01-28,Canvass,Out of Business,NaN,41.880396,-87.624502,2021
5301,2454105,NaN,0.0,Restaurant,Risk 2 (Medium),3407 1/2 W MADISON ST,CHICAGO,IL,60624.0,2020-11-02,Canvass,Out of Business,NaN,41.880805,-87.711375,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250938,229498,XANDO COFFEE & BAR / COSI SANDWICH BAR,62.0,Restaurant,Risk 1 (High),230 W WASHINGTON ST,CHICAGO,IL,60606.0,2010-06-28,Canvass Re-Inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.883318,-87.634769,2010
254608,60405,CASA CENTRAL LA POSADA,0.0,Restaurant,Risk 1 (High),,NaN,IL,NaN,2010-04-05,Canvass,Pass,37. TOILET ROOM DOORS SELF CLOSING: DRESSING R...,NaN,NaN,2010
255452,68320,NaN,0.0,Restaurant,Risk 1 (High),3522 E 100TH ST,CHICAGO,IL,60617.0,2010-03-02,Consultation,Fail,NaN,41.713977,-87.537917,2010
256045,120273,UPS Cafeteria,0.0,Restaurant,Risk 1 (High),1400 S JEFFERSON ST,CHICAGO,IL,60607.0,2010-01-06,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.863501,-87.642210,2010


In [29]:
bad_license_numbers[bad_license_numbers['AKA Name']=='SUBWAY']

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,year
8406,2293545,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-13,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.968506,-87.688338,2019
94598,2300450,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-28,Canvass,Out of Business,NaN,41.968506,-87.688338,2019
94955,2293900,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-20,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968506,-87.688338,2019


In [30]:
business_info[business_info['ADDRESS']=='4771 N LINCOLN AVE']

,DOING BUSINESS AS NAME,LICENSE ID,ADDRESS,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
121543,SUBWAY,2677482,4771 N LINCOLN AVE,47.0,NaN,47-,NaN,1006,Retail Food Establishment,2677482.0,781,Sale of Food Prepared Onsite With Dining Area,ISSUE,06/25/2019,N,07/24/2019,08/15/2021,07/22/2019,07/24/2019
416655,CAFE DESCARTES,1646813,4771 N LINCOLN AVE,47.0,20.0,47-20,19.0,1006,Retail Food Establishment,1646813.0,775,Retail Sales of Perishable Foods,ISSUE,10/21/2005,N,11/17/2005,11/15/2006,11/16/2005,11/17/2005
859424,CAFE DESCARTES,1747076,4771 N LINCOLN AVE,47.0,20.0,47-20,19.0,1006,Retail Food Establishment,1646813.0,775,Retail Sales of Perishable Foods,RENEW,09/20/2006,N,11/16/2006,11/15/2007,12/19/2006,12/19/2006
920645,SUBWAY,2796097,4771 N LINCOLN AVE,47.0,NaN,47-,NaN,1006,Retail Food Establishment,2677482.0,781,Sale of Food Prepared Onsite With Dining Area,RENEW,06/15/2021,N,08/16/2021,08/15/2023,01/31/2022,09/07/2022


In [31]:
# this should work

# name = (bad_license_numbers[bad_license_numbers['Inspection ID'] == 2576806]['Address']).values[0]

bad_license_numbers[bad_license_numbers['Address']=='4771 N LINCOLN AVE ']

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,year
8406,2293545,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-13,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.968506,-87.688338,2019
94598,2300450,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-28,Canvass,Out of Business,NaN,41.968506,-87.688338,2019
94955,2293900,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-20,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968506,-87.688338,2019


In [32]:
bad_license_numbers['Address'] = bad_license_numbers['Address'].apply(lambda x: x.strip())
bad_license_numbers[bad_license_numbers['Address']=='4771 N LINCOLN AVE']

C:\Users\ejwda\AppData\Local\Temp\ipykernel_26620\3100172670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_license_numbers['Address'] = bad_license_numbers['Address'].apply(lambda x: x.strip())


,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,year
8406,2293545,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-13,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.968506,-87.688338,2019
94598,2300450,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-28,Canvass,Out of Business,NaN,41.968506,-87.688338,2019
94955,2293900,SUBWAY,0.0,Restaurant,Risk 1 (High),4771 N LINCOLN AVE,CHICAGO,IL,60625.0,2019-06-20,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968506,-87.688338,2019


In [33]:
bad_license_numbers2 = business_info[business_info['LICENSE NUMBER'] < 100]
len(bad_license_numbers2)

109

### See if the correct License # can be obtained from business_info by matching addresses

In [34]:
business_info[business_info['DOING BUSINESS AS NAME'] == 'SUBWAY'].sort_values(by='ADDRESS', ascending=True)

,DOING BUSINESS AS NAME,LICENSE ID,ADDRESS,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
425501,SUBWAY,2619322,1 E 83RD ST 1,6.0,37.0,6-37,6.0,1006,Retail Food Establishment,1938179.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2018,N,10/16/2018,10/15/2020,10/09/2018,10/10/2018
358040,SUBWAY,2352753,1 E 83RD ST 1,6.0,37.0,6-37,6.0,1006,Retail Food Establishment,1938179.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2014,N,10/16/2014,10/15/2016,09/11/2014,09/12/2014
175225,SUBWAY,2746803,1 E 83RD ST 1,6.0,37.0,6-37,6.0,1006,Retail Food Establishment,1938179.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2020,N,10/16/2020,10/15/2022,08/17/2020,12/22/2020
896790,SUBWAY,2486131,1 E 83RD ST 1,6.0,37.0,6-37,6.0,1006,Retail Food Establishment,1938179.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2016,N,10/16/2016,10/15/2018,09/12/2016,09/13/2016
974323,SUBWAY,2865113,1 E 83RD ST 1,6.0,37.0,6-37,6.0,1006,Retail Food Establishment,1938179.0,775,Retail Sales of Perishable Foods,RENEW,08/15/2022,N,10/16/2022,10/15/2024,10/26/2022,03/13/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784155,SUBWAY,1846937,9518 S HALSTED ST 1ST,21.0,45.0,21-45,22.0,1006,Retail Food Establishment,1846937.0,775,Retail Sales of Perishable Foods,ISSUE,08/31/2007,N,10/24/2007,09/15/2009,10/23/2007,10/24/2007
233751,SUBWAY,2546362,9518 S HALSTED ST 1ST,21.0,45.0,21-45,22.0,1006,Retail Food Establishment,1846937.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2017,N,09/16/2017,09/15/2019,07/27/2017,07/28/2017
643665,SUBWAY,2275574,9518 S HALSTED ST 1ST,21.0,45.0,21-45,22.0,1006,Retail Food Establishment,1846937.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2013,N,09/16/2013,09/15/2015,08/20/2013,08/21/2013
332089,SUBWAY,1984256,9518 S HALSTED ST 1ST,21.0,45.0,21-45,22.0,1006,Retail Food Establishment,1846937.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2009,N,09/16/2009,09/15/2011,09/14/2009,09/15/2009


In [35]:
len(bad_license_numbers)

247

In [36]:
# most likely this does not work, as is, because year-to-year the restaurants change
temp = pd.merge(bad_license_numbers, business_info, left_on='Address', right_on='ADDRESS', how='left')
temp

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,...,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
0,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,05/15/2008,N,07/16/2008,07/15/2010,06/19/2008,06/20/2008
1,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/21/2005,N,08/16/2005,08/15/2006,08/19/2005,08/22/2005
2,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/21/2006,N,08/16/2006,08/15/2007,08/16/2006,08/17/2006
3,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,2900944.0,767,Retail Sales of General Merchandise and Non-Pe...,ISSUE,03/28/2023,N,03/28/2023,10/15/2023,03/28/2023,03/28/2023
4,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/30/2004,N,08/16/2004,08/15/2005,08/20/2004,08/23/2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12927,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2008,N,09/16/2008,09/15/2010,08/22/2008,08/25/2008
12928,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/16/2012,N,09/16/2012,09/15/2014,09/10/2012,09/11/2012
12929,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2014,N,09/16/2014,09/15/2016,10/02/2014,12/18/2014
12930,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,1093940.0,NaN,NaN,RENEW,09/24/2002,N,11/16/2002,11/15/2003,11/14/2002,11/15/2002


In [37]:
temp.dropna(subset='LICENSE NUMBER', axis=0, inplace=True)
# temp.set_index('Inspection ID', inplace=True)
# restaurant_df.set_index('Inspection ID', inplace=True)
temp

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,...,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
0,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,05/15/2008,N,07/16/2008,07/15/2010,06/19/2008,06/20/2008
1,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/21/2005,N,08/16/2005,08/15/2006,08/19/2005,08/22/2005
2,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/21/2006,N,08/16/2006,08/15/2007,08/16/2006,08/17/2006
3,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,2900944.0,767,Retail Sales of General Merchandise and Non-Pe...,ISSUE,03/28/2023,N,03/28/2023,10/15/2023,03/28/2023,03/28/2023
4,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/30/2004,N,08/16/2004,08/15/2005,08/20/2004,08/23/2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12927,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2008,N,09/16/2008,09/15/2010,08/22/2008,08/25/2008
12928,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/16/2012,N,09/16/2012,09/15/2014,09/10/2012,09/11/2012
12929,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2014,N,09/16/2014,09/15/2016,10/02/2014,12/18/2014
12930,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,1093940.0,NaN,NaN,RENEW,09/24/2002,N,11/16/2002,11/15/2003,11/14/2002,11/15/2002


In [38]:
restaurant_df

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,year
0,2579974,PING PONG,1765705.0,Restaurant,Risk 1 (High),3322-3326 N BROADWAY,CHICAGO,IL,60657.0,2023-08-11,Non-Inspection,No Entry,NaN,41.942623,-87.644625,2023
5,2579896,LAS ISLAS MARIAS,1943409.0,Restaurant,Risk 1 (High),2400 S PULASKI RD,CHICAGO,IL,60623.0,2023-08-10,Canvass Re-Inspection,Fail,NaN,41.847858,-87.724795,2023
6,2579853,Lito's Taco and Empanada House,2762609.0,Restaurant,Risk 1 (High),2460 N CLARK ST,CHICAGO,IL,60614.0,2023-08-09,Canvass Re-Inspection,No Entry,NaN,41.927067,-87.641530,2023
7,2579910,SHU TIN XIA,2574892.0,Restaurant,Risk 1 (High),2428-2436 S WALLACE ST,CHICAGO,IL,60616.0,2023-08-09,Canvass,Out of Business,NaN,41.848386,-87.641960,2023
8,2579839,QUERETACO,2918599.0,Restaurant,Risk 2 (Medium),2247 E 71ST ST,CHICAGO,IL,60649.0,2023-08-09,License,Pass,NaN,41.766032,-87.569550,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258044,63258,CHICKIES,1848186.0,Restaurant,Risk 2 (Medium),2839 S PULASKI RD,CHICAGO,IL,60623.0,2010-02-10,Complaint,Fail,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",41.839389,-87.724271,2010
258045,215227,WENDY'S,1145755.0,Restaurant,Risk 1 (High),7601 S CICERO AVE,CHICAGO,IL,60652.0,2010-02-22,Complaint-Fire Re-inspection,Pass,NaN,41.754660,-87.741385,2010
258049,114321,HOTEL CHICAGO,1680526.0,Restaurant,Risk 1 (High),333 N DEARBORN ST,CHICAGO,IL,60654.0,2010-02-04,Canvass,Pass w/ Conditions,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.888414,-87.629411,2010
258051,70292,EL GORDO RESTAURANT,31100.0,Restaurant,Risk 1 (High),5746 W BELMONT AVE,CHICAGO,IL,60634.0,2010-01-19,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938627,-87.770830,2010


In [39]:
temp


,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,...,LICENSE NUMBER,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,APPLICATION TYPE,APPLICATION REQUIREMENTS COMPLETE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED
0,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,05/15/2008,N,07/16/2008,07/15/2010,06/19/2008,06/20/2008
1,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/21/2005,N,08/16/2005,08/15/2006,08/19/2005,08/22/2005
2,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/21/2006,N,08/16/2006,08/15/2007,08/16/2006,08/17/2006
3,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,2900944.0,767,Retail Sales of General Merchandise and Non-Pe...,ISSUE,03/28/2023,N,03/28/2023,10/15/2023,03/28/2023,03/28/2023
4,2576806,MINI SUPER DANY,0.0,Restaurant,Risk 1 (High),10111 S EWING AVE,CHICAGO,IL,60617.0,2023-06-05,...,1144141.0,NaN,NaN,RENEW,06/30/2004,N,08/16/2004,08/15/2005,08/20/2004,08/23/2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12927,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2008,N,09/16/2008,09/15/2010,08/22/2008,08/25/2008
12928,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/16/2012,N,09/16/2012,09/15/2014,09/10/2012,09/11/2012
12929,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,99.0,775,Retail Sales of Perishable Foods,RENEW,07/15/2014,N,09/16/2014,09/15/2016,10/02/2014,12/18/2014
12930,197244,COSI,99.0,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,2010-02-03,...,1093940.0,NaN,NaN,RENEW,09/24/2002,N,11/16/2002,11/15/2003,11/14/2002,11/15/2002


In [41]:
# Need to re-index both dataframes

# restaurant_df['License #'] = restaurant_df.replace(restaurant_df['License #'], dict(temp['LICENSE NUMBER']), inplace=True)

In [40]:
restaurant_df[restaurant_df['AKA Name'] == 'SUBWAY'].sort_values(by='License #', ascending=True)
restaurant_df.fillna('-99.  Unknown', inplace=True)

In [41]:
restaurant_df['License #'].value_counts()

1354323.0    196
0.0          158
1574001.0     69
60184.0       60
1974745.0     58
            ... 
2827305.0      1
2771129.0      1
2293833.0      1
2283741.0      1
1800100.0      1
Name: License #, Length: 24083, dtype: int64

In [58]:
# Solution :  Remove License # under 100 becasue they are probably incorrect

In [42]:
restaurant_df['Results'].value_counts()

Pass                    88166
Fail                    32544
Pass w/ Conditions      29606
Out of Business         12351
No Entry                 8451
Not Ready                2062
Business Not Located       26
Name: Results, dtype: int64

In [43]:
temp = restaurant_df.groupby(['AKA Name', 'License #']).count().iloc[:,0:2]

names = temp.index.get_level_values(0)
temp

Inspection ID  Facility Type
AKA Name               License #                              
#1 CHINA EXPRESS , LTD 1869616.0              2              2
#1 CHOP SUEY           1970042.0             26             26
                       2042372.0             11             11
                       2137377.0             20             20
                       2695112.0              4              4
...                                         ...            ...
naansense              2098531.0              8              8
no signage             1292.0                 3              3
the FRENCH LUNCHBOX    2511081.0              5              5
vitino pizzeria        2009515.0              1              1
ï¿½ BAR AND GRILL      25257.0               22             22

[25046 rows x 2 columns]

In [44]:
len(temp.xs('1 N WACKER KITCHEN AND BAR', level='AKA Name'))


2

In [45]:
location_counts = []
for name in names:
    num = len(temp.xs(name, level='AKA Name'))
    location_counts.append(num)

temp['locations'] = location_counts
temp.drop(labels='Facility Type', axis=1, inplace=True)
temp = temp.sort_values(by=['locations'], ascending=False)

In [46]:
temp

Inspection ID  locations
AKA Name                      License #                          
-99.  Unknown                 2368190.0              5        388
                              2293822.0              8        388
                              2289715.0              1        388
                              2289533.0              3        388
                              2284667.0              7        388
...                                                ...        ...
KENTUCKY FRIED CHICKEN 521038 1816808.0             10          1
KENTUCKY FRIED CHICKEN 521039 1816810.0             11          1
KENTUCKY FRIED CHICKEN 521040 1816811.0             11          1
KENTUCKY FRIED CHICKEN 521041 1816812.0              6          1
ï¿½ BAR AND GRILL             25257.0               22          1

[25046 rows x 2 columns]

In [47]:
temp.xs('DUNKIN DONUTS', level='AKA Name')

,Inspection ID,locations
License #,,
2569804.0,6,188
1677626.0,8,188
2642143.0,8,188
2569844.0,3,188
2642263.0,1,188
...,...,...
1913870.0,14,188
1913868.0,24,188
2890954.0,1,188


In [48]:
restaurant_df['Violations_List'] = restaurant_df['Violations'].apply(lambda x: x.split("|"))
restaurant_df['Number_violations'] = restaurant_df['Violations_List'].apply(lambda x: len(x))
restaurant_df.head(2)

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,year,Violations_List,Number_violations
0,2579974,PING PONG,1765705.0,Restaurant,Risk 1 (High),3322-3326 N BROADWAY,CHICAGO,IL,60657.0,2023-08-11,Non-Inspection,No Entry,-99. Unknown,41.942623,-87.644625,2023,[-99. Unknown],1
5,2579896,LAS ISLAS MARIAS,1943409.0,Restaurant,Risk 1 (High),2400 S PULASKI RD,CHICAGO,IL,60623.0,2023-08-10,Canvass Re-Inspection,Fail,-99. Unknown,41.847858,-87.724795,2023,[-99. Unknown],1


In [49]:
import re

restaurant_df['violation_number'] = restaurant_df['Violations_List'].apply(lambda x: [re.findall(r'\b\d+\b',i)[0] for i in x])
restaurant_df.head(3)

restaurant_df['must_comply'] = restaurant_df['Violations_List'].apply(lambda x: [ 'MUST COMPLY' in i for i in x])
restaurant_df['must_comply_count'] = restaurant_df['must_comply'].apply(lambda x: sum(x))

restaurant_df['instructed_comply'] = restaurant_df['Violations_List'].apply(lambda x: ['INSTRUCTED TO COMPLY' in i for i in x])
restaurant_df['instructed_comply_count'] = restaurant_df['instructed_comply'].apply(lambda x: sum(x))

restaurant_df['not_cited'] = restaurant_df['Violations_List'].apply(lambda x: ['NO CITATION ISSUED' in i for i in x])
restaurant_df['citation_count'] = restaurant_df['not_cited'].apply(lambda x: len(x) - sum(x))

restaurant_df.head(2)

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,...,year,Violations_List,Number_violations,violation_number,must_comply,must_comply_count,instructed_comply,instructed_comply_count,not_cited,citation_count
0,2579974,PING PONG,1765705.0,Restaurant,Risk 1 (High),3322-3326 N BROADWAY,CHICAGO,IL,60657.0,2023-08-11,...,2023,[-99. Unknown],1,[99],[False],0,[False],0,[False],1
5,2579896,LAS ISLAS MARIAS,1943409.0,Restaurant,Risk 1 (High),2400 S PULASKI RD,CHICAGO,IL,60623.0,2023-08-10,...,2023,[-99. Unknown],1,[99],[False],0,[False],0,[False],1


In [50]:
temp = restaurant_df.groupby('License #').count()['AKA Name'].sort_values(ascending=False)
temp[temp > 1]

License #
1354323.0    196
0.0          158
1574001.0     69
60184.0       60
1974745.0     58
            ... 
1933087.0      2
2601610.0      2
2535860.0      2
2464561.0      2
2500745.0      2
Name: AKA Name, Length: 20082, dtype: int64

In [51]:
len(restaurant_df)

173206

In [52]:
temp = restaurant_df.sort_values(by=['Inspection Date']).groupby('License #')['Inspection Date'].apply(list)
temp2 = temp.apply(lambda x: len(x))
df = pd.concat([temp,temp2], axis=1)
df.columns=['Inspection Date', 'Total_inspections']  


df = df.sort_values(by=['Total_inspections'], ascending=False)
df['Date_min'] = df['Inspection Date'].apply(lambda x: min(x))
df['Date_max'] = df['Inspection Date'].apply(lambda x: max(x))



df.head(10)

,Inspection Date,Total_inspections,Date_min,Date_max
License #,,,,
1354323.0,"[2010-06-07 00:00:00, 2010-06-07 00:00:00, 201...",196,2010-06-07,2013-11-15
0.0,"[2010-01-06 00:00:00, 2010-03-02 00:00:00, 201...",158,2010-01-06,2023-07-19
1574001.0,"[2010-04-05 00:00:00, 2010-04-05 00:00:00, 201...",69,2010-04-05,2017-08-10
60184.0,"[2010-01-19 00:00:00, 2010-03-15 00:00:00, 201...",60,2010-01-19,2023-07-19
1974745.0,"[2011-02-08 00:00:00, 2011-02-08 00:00:00, 201...",58,2011-02-08,2017-11-07
39623.0,"[2010-01-07 00:00:00, 2010-01-14 00:00:00, 201...",50,2010-01-07,2023-05-08
1273271.0,"[2010-06-07 00:00:00, 2010-07-14 00:00:00, 201...",49,2010-06-07,2023-06-20
14616.0,"[2010-03-25 00:00:00, 2010-03-25 00:00:00, 201...",49,2010-03-25,2011-08-31
4190.0,"[2010-02-25 00:00:00, 2010-06-16 00:00:00, 201...",47,2010-02-25,2023-07-24


In [53]:
restaurant_df[restaurant_df['License #']==2637249]

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,...,year,Violations_List,Number_violations,violation_number,must_comply,must_comply_count,instructed_comply,instructed_comply_count,not_cited,citation_count
9506,2252616,CASA DEL SOL,2637249.0,Restaurant,Risk 1 (High),8918 S COMMERCIAL AVE,CHICAGO,IL,60617.0,2019-01-15,...,2019,"[3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...",18,"[3, 5, 25, 29, 32, 36, 36, 38, 39, 48, 51, 54,...","[False, False, False, False, False, False, Fal...",0,"[False, False, False, False, False, False, Fal...",0,"[True, True, True, False, False, True, False, ...",13
79930,2355665,CASA DEL SOL,2637249.0,Restaurant,Risk 1 (High),8918 S COMMERCIAL AVE,CHICAGO,IL,60617.0,2019-12-19,...,2019,[-99. Unknown],1,[99],[False],0,[False],0,[False],1
98305,2268587,CASA DEL SOL,2637249.0,Restaurant,Risk 1 (High),8918 S COMMERCIAL AVE,CHICAGO,IL,60617.0,2019-02-26,...,2019,"[3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...",4,"[3, 5, 57, 58]","[False, False, False, False]",0,"[False, False, False, False]",0,"[False, False, False, False]",4
98884,2243833,CASA DEL SOL,2637249.0,Restaurant,Risk 1 (High),8918 S COMMERCIAL AVE,CHICAGO,IL,60617.0,2018-12-26,...,2018,"[3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...",18,"[3, 5, 25, 29, 32, 36, 36, 38, 39, 48, 51, 54,...","[False, False, False, False, False, False, Fal...",0,"[False, False, False, False, False, False, Fal...",0,"[True, True, True, False, False, True, False, ...",13
101149,2253115,CASA DEL SOL,2637249.0,Restaurant,Risk 1 (High),8918 S COMMERCIAL AVE,CHICAGO,IL,60617.0,2019-01-25,...,2019,"[3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...",8,"[3, 5, 25, 29, 32, 36, 57, 58]","[False, False, False, False, False, False, Fal...",0,"[False, False, False, False, False, False, Fal...",0,"[True, True, True, False, False, True, False, ...",4
101370,2252398,CASA DEL SOL,2637249.0,Restaurant,Risk 1 (High),8918 S COMMERCIAL AVE,CHICAGO,IL,60617.0,2019-01-11,...,2019,"[3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL ...",18,"[3, 5, 25, 29, 32, 36, 36, 38, 39, 48, 51, 54,...","[False, False, False, False, False, False, Fal...",0,"[False, False, False, False, False, False, Fal...",0,"[True, True, True, False, False, True, False, ...",13


### Check for inspections occuring on the same day
If small then leave as is
If large then drop all duplicates except the last one

In [55]:
restaurant_df.columns

Index(['Inspection ID', 'AKA Name', 'License #', 'Facility Type', 'Risk',
       'Address', 'City', 'State', 'Zip', 'Inspection Date', 'Inspection Type',
       'Results', 'Violations', 'Latitude', 'Longitude', 'year',
       'Violations_List', 'Number_violations', 'violation_number',
       'must_comply', 'must_comply_count', 'instructed_comply',
       'instructed_comply_count', 'not_cited', 'citation_count'],
      dtype='object')

In [56]:
# Are there multiple inspections for the same day for the same license number? 
restaurant_df.groupby(['Inspection Date','License #', 'AKA Name']).count()['Inspection ID'].sort_values(ascending=False)

Inspection Date  License #  AKA Name             
2013-11-14       1354323.0  GRIDIRON GRILL           7
2010-06-07       1354323.0  GRIDIRON GRILL           5
2010-03-22       1621492.0  SUBWAY                   3
2010-06-07       1354323.0  2ND CITY PIZZA           3
2011-05-23       1354323.0  GRIDIRON GRILL           3
                                                    ..
2014-09-15       1579961.0  EL ENCANTO MICHOACANO    1
                 1650274.0  JOY YEE NOODLE           1
                 1740448.0  RESTAURANT SANTIAGO      1
                 1769269.0  MCDONALD'S               1
2023-08-11       2924439.0  TEPALCATES               1
Name: Inspection ID, Length: 172515, dtype: int64

In [57]:
# # drop duplicates ignoring the inspection id
# columns = list(restaurant_df.columns[1:])

# not including lists due to error, not including inspect_type because multiple complaints of different kinds could be submitted
restaurant_df.drop_duplicates(subset=['AKA Name', 'License #', 'Facility Type', 'Risk',
       'Address', 'City', 'State', 'Zip', 'Inspection Date', 'Results', 'Latitude', 'Longitude'], 
                              keep='last',
                              inplace=True)

Check license number consistency

In [59]:
# Is the company and address the same but different license number?
temp = restaurant_df.groupby(['AKA Name','Address','License #']).count()[['Inspection ID','Zip']].sort_values(by='Inspection ID', ascending=False)
temp

,,,Inspection ID,Zip
AKA Name,Address,License #,,
TAQUERIA EL RANCHITO,2829 N MILWAUKEE AVE,60184.0,60,60
LUMES PANCAKE HOUSE,11601 S WESTERN AVE,39623.0,50,50
CHURCH'S CHICKEN # 982,58 W 79TH ST,1273271.0,49,49
STAR OF SIAM,11 E ILLINOIS ST,4190.0,47,47
LAS ISLAS MARIAS,4770 W GRAND AVE,1095992.0,46,46
...,...,...,...,...
TASTE FOOD AND WINE,1506 W JARVIS AVE,2684138.0,1,1
BIRRERIA & TAQUERIA EL CARMEN,4101 W BELMONT AVE,1355490.0,1,1
LA COCINA DE FRIDA,5403 N CLARK ST,1823603.0,1,1


In [60]:
temp[temp.index.get_level_values('AKA Name') == 'SAPORI']

Inspection ID  Zip
AKA Name Address            License #                    
SAPORI   2701 N HALSTED ST  2320734.0             18   18
                            2098866.0              5    5
                            1124199.0              3    3
                            2098867.0              1    1
                            2320735.0              1    1

In [61]:
# two methods that are the same
#temp.loc[('SAPORI', slice(None), slice(None))]
subset_temp = temp.xs('SAPORI', level='AKA Name')
subset_temp

Inspection ID  Zip
Address            License #                    
2701 N HALSTED ST  2320734.0             18   18
                   2098866.0              5    5
                   1124199.0              3    3
                   2098867.0              1    1
                   2320735.0              1    1

In [62]:
# counts number of licence_num per business name
t4 = subset_temp.groupby(level='Address').count()['Inspection ID']
t4.values[0]

5

In [63]:
# number of businesses per address
t = temp.groupby(level='Address').count().sort_values(by='Inspection ID', ascending=False)['Inspection ID']
t.index[0]

# Note:  11601 W TOUHY AVE I think represents the entire O'Hare Airport

'11601 W TOUHY AVE '

In [64]:
location_counts = []

names = temp.index.get_level_values(0)
for name in names:
    num = len(temp.xs(name, level='AKA Name'))
    location_counts.append(num)

# temp2 = pd.Series(location_counts, names)
# pd.DataFrame({temp, temp}])
temp['duplicate_licenses'] = location_counts
temp.sort_values(by='duplicate_licenses', ascending=False)

Inspection ID  \
AKA Name                Address                  License #                  
-99.  Unknown           5025 W Belmont AVE       0.0                    1   
                        11346 S MICHIGAN AVE     2027121.0              3   
                        658 E 75th ST            1976054.0              2   
                        2751 W 47TH ST           2141164.0              2   
                        2759 S AVERS AVE         2163196.0              2   
...                                                                   ...   
COFFEE LAB              2823 N LINCOLN AVE       2615723.0              5   
EL VENEDO DE LA VILLITA 3857 W 26TH ST           2398096.0              5   
DINICO'S PIZZA II       2341 N NARRAGANSETT AVE  2766818.0              5   
KYU / SSAM              939 N ASHLAND AVE        2451473.0              5   
ROOM 21                 2110 S WABASH AVE        1800726.0              1   

                                                            Zip  \
AKA Name                Address                  License #        
-99.  Unknown           5025 W Belmont AVE       0.0          1   
                        11346 S MICHIGAN AVE     2027121.0    3   
                        658 E 75th ST            1976054.0    2   
                        2751 W 47TH ST           2141164.0    2   
                        2759 S AVERS AVE         2163196.0    2   
...                                                         ...   
COFFEE LAB              2823 N LINCOLN AVE       2615723.0    5   
EL VENEDO DE LA VILLITA 3857 W 26TH ST           2398096.0    5   
DINICO'S PIZZA II       2341 N NARRAGANSETT AVE  2766818.0    5   
KYU / SSAM              939 N ASHLAND AVE        2451473.0    5   
ROOM 21                 2110 S WABASH AVE        1800726.0    1   

                                                            duplicate_licenses  
AKA Name                Address                  License #                      
-99.  Unknown           5025 W Belmont AVE       0.0                       400  
                        11346 S MICHIGAN AVE     2027121.0                 400  
                        658 E 75th ST            1976054.0                 400  
                        2751 W 47TH ST           2141164.0                 400  
                        2759 S AVERS AVE         2163196.0                 400  
...                                                                        ...  
COFFEE LAB              2823 N LINCOLN AVE       2615723.0                   1  
EL VENEDO DE LA VILLITA 3857 W 26TH ST           2398096.0                   1  
DINICO'S PIZZA II       2341 N NARRAGANSETT AVE  2766818.0                   1  
KYU / SSAM              939 N ASHLAND AVE        2451473.0                   1  
ROOM 21                 2110 S WABASH AVE        1800726.0                   1  

[25104 rows x 3 columns]

In [65]:
restaurant_df[restaurant_df['Address']=='11601 W TOUHY AVE']

,Inspection ID,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,...,year,Violations_List,Number_violations,violation_number,must_comply,must_comply_count,instructed_comply,instructed_comply_count,not_cited,citation_count
